# <span style='color:red'>Project 2.  Due October 23</span>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

C:\Users\Guido De Filippo\AppData\Local\Temp\ipykernel_17444\1189900070.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### In this project we develop a first-order algorithm to construct a portfolio using intraday data.

In [2]:
import csv
import sys
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt

##### We will have data involving $n$ assets, and use the first $T$ days of the data to compute the portfolio.
##### The computation will produce a weight $x_i$ for each asset $i = 1,...,n$, which could be long or short.
##### We assume that on each day, a position is taken at the open, and closed at noon.  So we define:
$$ p^o_{j,t} = \ \text{price of asset $j$ on day $t$ at the open}$$
$$ p^1_{j,t} = \ \text{price of asset $j$ on day $t$ at noon}$$
$$ r_{j,t} =  \ \frac{p^1_{j,t} - p^o_{j,t}}{p^o_{j,t}} = \ \text{return earned by asset $j$ on day $t$.}$$
$$ \bar r_j = \ \frac{1}{T} \sum_{t = 1}^T r_{j,t} = \ \text{average return earned by asset $j$.}$$

#### The optimization problem to solve depends on two parameters: $\theta \ge 0$ and $\pi > 0 0$.
####
$$ \text{minimize} \ \left(-\sum_{j = 1}^n \bar r_j x_j\right) \ + \ \theta \left( \frac{1}{T} \sum_{t = 1}^{T}\left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^\pi\right)^{1/\pi}$$
#### 
#### There are no constraints on the quantities $x_j$.
#### The first sum is minus the average return earned by the portfolio.  In the second sum, the quantity inside the square brackets is the excess return earned by the portfolio on day $t$, magnified by the power $\pi$.  The quantity $\theta$ is a risk aversion parameter.
 

### <span style='color:red'> Task 1. Develop a first-order method to address this computational problem.</span>
#### 
#### Your method should work with values of $T$ at least $100$. Use the data we provide for AMZN, NFLX, TSLA, i.e., $n = 3$. 
###
#### Make sure your code works with $\pi = 0.5, 2, 4, 6$, and $\theta = 0.1, 10, 1000, 10^5, 10^6$.

Denote the objective function as  $F(x)$. We will take the gradient of $ F(x) $ with respect to the vector $ x $. The gradient will be a vector of partial derivatives, where each element $ k $ is the partial derivative of $ F(x) $ with respect to $ x_k $.

$$ F(x) = \left(-\sum_{j = 1}^n \bar r_j x_j\right) + \theta \left( \frac{1}{T} \sum_{t = 1}^{T}\left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^\pi\right)^{1/\pi} $$

Let's take the derivative piece by piece.

1. **Derivative of the first term**:
$$ \frac{\partial}{\partial x_k} \left(-\sum_{j = 1}^n \bar r_j x_j\right) = -\bar r_k $$

2. **Derivative of the second term**:
For the second term, we'll need to apply the chain rule and the formula for differentiating powers. The outer function is $ g(u) = u^{1/\pi} $ and the inner function is $ u = \frac{1}{T} \sum_{t = 1}^{T}\left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^\pi $.

We first find $ \frac{du}{dx_k} $ and then use the chain rule to find $ \frac{dg}{dx_k} $:

$$ \frac{du}{dx_k} = \frac{1}{T} \sum_{t = 1}^{T} \pi \left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^{\pi-1}(r_{k,t} -  \bar r_k) $$

Now apply the chain rule:

$$ \frac{dg}{dx_k} = \frac{d}{du} g(u) \cdot \frac{du}{dx_k} = \frac{1}{\pi} u^{(1/\pi)-1} \cdot \frac{du}{dx_k} $$

Now, putting it all together for the second term, we get:

$$ \theta \cdot \frac{dg}{dx_k} = \theta \cdot \frac{1}{\pi} u^{(1/\pi)-1} \cdot \frac{1}{T} \sum_{t = 1}^{T} \pi \left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^{\pi-1}(r_{k,t} -  \bar r_k) $$

3. **Sum of the derivatives**:
Now summing the derivatives of the two terms, we obtain the $ k $-th component of the gradient:

$$ \frac{\partial F(x)}{\partial x_k} = -\bar r_k + \theta \cdot \frac{1}{\pi} u^{(1/\pi)-1} \cdot \frac{1}{T} \sum_{t = 1}^{T} \pi \left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^{\pi-1}(r_{k,t} -  \bar r_k) $$

In [3]:
def compute_gradient(x, r, r_bar, theta, pi, T):
    n = len(x)
    gradient = np.zeros(n)
    
    # Compute u
    u = (1/T) * np.sum([(np.sum([(r[t, j] - r_bar[j]) * x[j] for j in range(n)]))**pi for t in range(T)])
    
    for k in range(n):
        term1 = -r_bar[k]
        term2 = theta * (1/pi) * u**((1/pi)-1) * (1/T) * np.sum([pi * (np.sum([(r[t, j] - r_bar[j]) * x[j] for j in range(n)]))**(pi-1) * (r[t, k] - r_bar[k]) for t in range(T)])
        gradient[k] = term1 + term2
        
    return gradient



In [4]:
def gradient_descent(r, r_bar, initial_x, alpha, theta, pi, T, max_iterations=1000, tolerance=1e-6):
    x = initial_x
    for k in range(max_iterations):
        gradient = compute_gradient(x, r, r_bar, theta, pi, T)
        
        # Update rule
        x_new = x - alpha * gradient
        
        # Convergence check
        if np.linalg.norm(x_new - x) < tolerance:
            break
        
        x = x_new
        
    return x

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load processed returns
tickers = ['AMZN', 'NFLX', 'TSLA']

# Load processed returns as DataFrames
returns_df = {}
for ticker in tickers:
    returns_df[ticker] = pd.read_csv(f'./data/{ticker}_processed.csv', index_col=0)

# Find the common dates where all tickers have returns
common_dates = set(returns_df[tickers[0]].index)
for ticker in tickers[1:]:
    common_dates.intersection_update(set(returns_df[ticker].index))

# Filter the returns for each ticker based on the common dates
for ticker in tickers:
    returns_df[ticker] = returns_df[ticker].loc[common_dates]

# Convert filtered returns to matrix form
r = np.column_stack([returns_df[ticker]['Returns'].values for ticker in tickers])

r_train = r[:100]
r_test = r[100:]

C:\Users\Guido De Filippo\AppData\Local\Temp\ipykernel_17444\1716008853.py:20: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  returns_df[ticker] = returns_df[ticker].loc[common_dates]
C:\Users\Guido De Filippo\AppData\Local\Temp\ipykernel_17444\1716008853.py:20: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  returns_df[ticker] = returns_df[ticker].loc[common_dates]
C:\Users\Guido De Filippo\AppData\Local\Temp\ipykernel_17444\1716008853.py:20: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  returns_df[ticker] = returns_df[ticker].loc[common_dates]


In [21]:
# Compute average returns
r_bar = np.mean(r_train, axis=0)

# Gradient Descent parameters
initial_x = np.ones(3) / 3  # equally weighted portfolio
alpha = 0.01  # step size, might need adjustment
pis = [0.5, 2, 4, 6]
thetas = [0.1, 10, 1000, 10**5, 10**6]

# Run Gradient Descent for each combination of pi and theta
results = {}
for pi in pis:
    for theta in thetas:
        x_optimal = gradient_descent(r_train, r_bar, initial_x, alpha, theta, pi, r_train.shape[0])
        results[(pi, theta)] = x_optimal

#print(results)

C:\Users\Guido De Filippo\AppData\Local\Temp\ipykernel_17444\3891682047.py:6: RuntimeWarning: invalid value encountered in double_scalars
  u = (1/T) * np.sum([(np.sum([(r[t, j] - r_bar[j]) * x[j] for j in range(n)]))**pi for t in range(T)])
C:\Users\Guido De Filippo\AppData\Local\Temp\ipykernel_17444\3891682047.py:10: RuntimeWarning: invalid value encountered in double_scalars
  term2 = theta * (1/pi) * u**((1/pi)-1) * (1/T) * np.sum([pi * (np.sum([(r[t, j] - r_bar[j]) * x[j] for j in range(n)]))**(pi-1) * (r[t, k] - r_bar[k]) for t in range(T)])


In [7]:
results

{(0.5, 0.1): array([nan, nan, nan]),
 (0.5, 10): array([nan, nan, nan]),
 (0.5, 1000): array([nan, nan, nan]),
 (0.5, 100000): array([nan, nan, nan]),
 (0.5, 1000000): array([nan, nan, nan]),
 (2, 0.1): array([0.33089214, 0.32279751, 0.29718623]),
 (2, 10): array([0.00014672, 0.00013155, 0.00055289]),
 (2, 1000): array([0.01404502, 0.01619945, 0.06540541]),
 (2, 100000): array([1.40798618, 1.62784273, 6.57029954]),
 (2, 1000000): array([18.27827835, 21.13288904, 85.29632295]),
 (4, 0.1): array([0.32734281, 0.32201514, 0.28701329]),
 (4, 10): array([-0.00075146, -0.00055599, -0.00293777]),
 (4, 1000): array([-0.05148531, -0.03710566, -0.19790052]),
 (4, 100000): array([2.23584687, 1.60966228, 8.58706772]),
 (4, 1000000): array([ 32.35698151,  23.29531792, 124.27310172]),
 (6, 0.1): array([0.32392873, 0.32254587, 0.27542273]),
 (6, 10): array([0.00114648, 0.00058826, 0.00363312]),
 (6, 1000): array([0.0781723 , 0.04208522, 0.25984262]),
 (6, 100000): array([1.40278527, 0.75646427, 4.6700

### <span style='color:red'>Task 2: Benchmark your portfolio on the remaining days</span>
#### On each of the remaining days, we proceed as follows.  Denote by $x^*$ your portfolio. At the market open we invest $10^9 x^*_j$ on each asset $j$, and we close the position (by) noon.  You need to use the asset's price to compute the number of shares that you invest in, whether long or short. So the total you invest equals $$ \sum_{j = 1}^n 10^9 |x^*_j|.$$
#### Report the average return earned by your portfolio.

In [15]:
len(r[100:])

11

In [9]:
# Assuming x_star is the optimal portfolio obtained from the Gradient Descent
chosen_pi = 2
chosen_theta = 100000
x_star = results[(chosen_pi, chosen_theta)]  # replace chosen_pi and chosen_theta with the values you used



# Compute the total investment and returns for each day
total_investments = []
total_returns = []
for t in range(len(remaining_data[tickers[0]])):
    daily_investment = 0
    daily_return = 0
    for j, ticker in enumerate(tickers):
        open_price = remaining_data[ticker].iloc[t]['Open']
        noon_price = remaining_data[ticker].iloc[t]['Close']
        
        # Number of shares for asset j
        shares = (10**9 * x_star[j]) / open_price
        
        # Total investment for asset j
        daily_investment += abs(10**9 * x_star[j])
        
        # Return for asset j
        asset_return = (noon_price - open_price) * shares
        daily_return += asset_return
        
    total_investments.append(daily_investment)
    total_returns.append(daily_return)

# Compute the average return
average_return = np.mean(total_returns) / np.mean(total_investments)

print(f"Average return earned by the portfolio: {average_return:.2%}")


FileNotFoundError: [Errno 2] No such file or directory: './data/AMZN_remaining.csv'

In [8]:
def compute_portfolio_return(x_star, r, po, p1):
    n = len(x_star)
    daily_returns = []
    
    for t in range(len(r)):
        shares = [10**9 * x_star[j] / po[t, j] for j in range(n)]
        daily_return = sum([shares[j] * (p1[t, j] - po[t, j]) for j in range(n)])
        daily_returns.append(daily_return)
    
    return np.mean(daily_returns)

# Split the data into training and testing sets
T_train = T  # Number of training days
po_train = po[:T_train]  # Prices at open for training days
p1_train = p1[:T_train]  # Prices at noon for training days

po_test = po[T_train:]  # Prices at open for testing days
p1_test = p1[T_train:]  # Prices at noon for testing days

# Compute the average return for the portfolio on the testing days
average_return = compute_portfolio_return(x_optimal, r[T_train:], po_test, p1_test)
print(f"Average return on the remaining days: {average_return}")


NameError: name 'T' is not defined